## Use SX5E and bloomberg download data to get me started

In [417]:
import pandas as pd
import numpy as np
from datetime import datetime


In [436]:
#initial cleaning function
def clean_sheet(dataframe):
    adj=dataframe.dropna(axis=1,how='all')
    adj=dataframe.set_index(adj.columns[0]) #use stock tickers as the index
    return adj

In [437]:
#pull the data from excel, form a dictionary of tab names and dataframes
tabs_to_grab=['Quarter','Date','Reported EPS','Comp EPS','Consensus EPS','50d MA','3d return','Volume','Momentum',
             'Revision','Market cap','PE','property','Price']

data_dict={}

for s in tabs_to_grab:
    data_dict[s]=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Post result code\Catalyst.xlsx',sheet_name=s))


In [438]:
#signal formation and use the date as the base dataframe
signal=data_dict['Date'][(data_dict['3d return']>2) & (data_dict['Revision']>0.02)]
signal=signal.dropna(axis=1,how='all')
signal=signal.dropna(axis=0,how='all')

In [441]:
#Stack signals into a list of tuples (stocks, dates), convert dates into a common format of datetime
signal_list=[]
for index, row in signal.iterrows():
    adj_row=row.dropna()
    for s in adj_row.values:
        ts=(s - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's') 
        adj_time=datetime.utcfromtimestamp(ts).strftime("%d/%b/%Y") #transform the datetime into a common format
        signal_list.append((adj_row.name,adj_time))

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  


In [442]:
# grab the price time series and transform it into the same datetime format
price=data_dict["Price"]
new_column=pd.to_datetime(price.columns).strftime("%d/%b/%Y").tolist()
price.columns=new_column

In [443]:
# create a dictionary to have signal information (tuple) as keys and return data time series as values to make it easier to check
#; replace datetime with number of days post announcement to make it more comparable 
def signal_return_dict(signal_list,price_dataframe,holding):
    signal_dict={}
    for s in signal_list:
        price_series=price_dataframe.loc[s[0]]
        return_series=price_series.diff(1)/price_series
        day0=return_series.index.tolist().index(s[1]) #index of day 0
        target_series=return_series.iloc[day0:day0+min(holding, len(return_series[day0:]))]
        target_series.index=range(min(holding, len(return_series[day0:]))) #replace datetime with number of days post announcement
        target_series.name=s
        signal_dict[s]=target_series

    return signal_dict

return_dict=signal_return_dict(signal_list,price,63)

In [408]:
# obtain the dataframe of return time series for the signal stocks
return_series_list=[]
for index,key in return_dict.items():
    return_series_list.append(key)
    
return_df=pd.concat(return_series_list,axis=1)
return_df=return_df.dropna(how='any',axis=1) #drop stocks with limited history to make it more comparable

In [412]:
return_df.cumsum(axis=0)

ADS GY      ALV GY     ASML NA                  BAS GY      BMW GY  \
   08/Mar/2017 16/Feb/2017 19/Jul/2017 19/Oct/2016 24/Oct/2017 03/Aug/2017   
0     0.086074   -0.002526    0.054700    0.021609   -0.009877    0.006551   
1     0.104046    0.024808    0.072111    0.015351    0.000447    0.030767   
2     0.111848    0.032428    0.068311    0.035309    0.017398    0.026446   
3     0.114349    0.040890    0.069070    0.037694    0.030294    0.023847   
4     0.089862    0.046897    0.067550    0.031853    0.028051    0.014855   
5     0.112134    0.042068    0.073972    0.030913    0.028051    0.014605   
6     0.130981    0.030465    0.078110    0.032373    0.052944    0.013354   
7     0.131800    0.028629    0.054622    0.035698    0.052736    0.016471   
8     0.122155    0.033197    0.041356    0.038392    0.063250    0.013971   
9     0.118005    0.053776    0.039793    0.015388    0.067867    0.015718   
10    0.108507    0.051983    0.032709    0.009308    0.058965    0.011583   
11    0.111015    0.052580    0.042452   -0.005630    0.059689    0.008189   
12    0.105974    0.051684    0.054389   -0.005630    0.043607    0.000971   
13    0.091774    0.057921    0.063546    0.015872    0.036088    0.006136   
14    0.102451    0.057921    0.065450    0.015660    0.029587    0.002723   
15    0.102451    0.067626    0.058548    0.015024    0.021203    0.011245   
16    0.101607    0.062601    0.048869    0.001704    0.015691    0.004939   
17    0.104412    0.066723    0.033938   -0.019132    0.016123    0.003929   
18    0.090479    0.062584    0.046738   -0.005503    0.010037   -0.007434   
19    0.100333    0.064649    0.054437   -0.001195    0.015979   -0.006796   
20    0.088943    0.071970    0.060177    0.008194    0.026773   -0.011540   
21    0.094606    0.069916    0.054791    0.031127    0.017170    0.001118   
22    0.104140    0.068152    0.048207    0.057515    0.014249    0.002761   
23    0.099067    0.069033    0.041186    0.056397    0.042535    0.019780   
24    0.089966    0.061040    0.059187    0.062055    0.023788    0.035191   
25    0.099264    0.075051    0.055728    0.067281    0.031758    0.045959   
26    0.102634    0.064729    0.052644    0.065973    0.026738    0.044384   
27    0.102634    0.065024    0.051872    0.053852    0.030675    0.048367   
28    0.102634    0.076391    0.042920    0.050172    0.015994    0.057338   
29    0.102072    0.080455    0.045637    0.047404    0.033597    0.062929   
..         ...         ...         ...         ...         ...         ...   
33    0.137078    0.086154    0.059308    0.025677    0.031160    0.070488   
34    0.144348    0.079472    0.061219    0.025258    0.028816    0.067528   
35    0.142460    0.085536    0.063506    0.025049    0.032532    0.075515   
36    0.137035    0.082059    0.078897    0.045170    0.031894    0.074927   
37    0.134588    0.078569    0.078521    0.042700    0.027628    0.071271   
38    0.134588    0.070952    0.094775    0.043009    0.026774    0.072566   
39    0.130219    0.075327    0.090695    0.053192    0.045525    0.073508   
40    0.125004    0.071520    0.092177    0.051151    0.033009    0.066993   
41    0.133981    0.071520    0.097701    0.081334    0.020661    0.083421   
42    0.135881    0.071520    0.098438    0.090648    0.026638    0.087251   
43    0.126287    0.062360    0.121099    0.122447    0.024713    0.087251   
44    0.134174    0.060880    0.132831    0.124342    0.024713    0.113471   
45    0.116178    0.060584    0.118040    0.125761    0.024713    0.116064   
46    0.107240    0.068513    0.114419    0.123390    0.025568    0.119882   
47    0.091349    0.096490    0.120537    0.127169    0.016837    0.117857   
48    0.097553    0.098484    0.107043    0.127169    0.005501    0.112767   
49    0.097271    0.098484    0.109589    0.133272    0.005501    0.112428   
50    0.077714    0.092753    0.135112    0.137945    0.000792    0.105937   
51    0.080582    0.094469    

In [416]:
return_df.columns[2]


('ASML NA', '19/Jul/2017')